## PyTortch Regression

The following example will:

* Set the input and output schemas.
* Upload a SKLearn Logistic Regression model to Wallaroo.
* Deploy a pipeline with the uploaded SKLearn model as a pipeline step.
* Perform a test inference.
* Undeploy the pipeline.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

wl = wallaroo.Client(auth_type="sso", interactive=True)

In [2]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

prefix = "sklearn-kmeans"

In [3]:
workspace = get_workspace(f"{prefix}-jch")
wl.set_current_workspace(workspace)

{'name': 'sklearn-kmeans-jch', 'id': 43, 'archived': False, 'created_by': '3cc9e92a-fa3c-4371-a7a7-487884df059e', 'created_at': '2023-06-15T21:45:49.756082+00:00', 'models': [], 'pipelines': []}

## Data & Model Creation

In [4]:
input_schema = pa.schema([
    pa.field('input', pa.list_(pa.float64(), list_size=10))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float64(), list_size=1))
])

## Upload model

In [5]:
model = wl.upload_model('pt-single-io-model', "./models/model-auto-conversion_pytorch_single_io_model.pt", framework=Framework.PYTORCH, input_schema=input_schema, output_schema=output_schema)
model

Waiting for model conversion... It may take up to 10.0min.
Model is Pending conversion..Converting.Pending conversion..Converting.......Ready.


{'name': 'sklearn-kmeans', 'version': '2acfa360-cc80-46f8-b447-9e368c90c72f', 'file_name': 'model-auto-conversion_sklearn_kmeans.pkl', 'image_path': 'proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mlflow-deploy:v2023.3.0-main-3367', 'last_update_time': datetime.datetime(2023, 6, 15, 21, 46, 45, 245248, tzinfo=tzutc())}

## Configure model and pipeline

In [6]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [7]:
pipeline_name = f"{prefix}-pipeline"
pipeline = wl.build_pipeline(pipeline_name)
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

Waiting for deployment - this will take up to 90s ............. ok


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.0.232',
   'name': 'engine-9b9b8d9d5-qn5c9',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'sklearn-kmeans-pipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'sklearn-kmeans',
      'version': '2acfa360-cc80-46f8-b447-9e368c90c72f',
      'sha': 'b378a614854619dd573ec65b9b4ac73d0b397d50a048e733d96b68c5fdbec896',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.1.54',
   'name': 'engine-lb-584f54c899-8xknx',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.244.0.231',
   'name': 'engine-sidekick-sklearn-kmeans-36-6b9d658c94-2mpxt',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

## Inference

In [8]:
mock_inference_data = np.random.rand(10, 10)
mock_dataframe = pd.DataFrame({"input": mock_inference_data.tolist()})
mock_json = mock_dataframe.to_json('data/pytorch-single-io-data.json', orient="records")

pipeline.infer_from_file('./data/single_io_data.json')

,time,in.petal length (cm),in.petal width (cm),in.sepal length (cm),in.sepal width (cm),out.output,check_failures
0,2023-06-15 21:47:03.761,1.4,0.2,5.1,3.5,0.0,0
1,2023-06-15 21:47:03.761,1.4,0.2,4.9,3.0,0.0,0


In [9]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,sklearn-kmeans-pipeline
created,2023-06-15 21:46:50.172884+00:00
last_updated,2023-06-15 21:46:50.204499+00:00
deployed,False
tags,
versions,"87efe5a0-e87e-4b97-9fde-fe68e840c2e8, 4d28c002-f5b8-4566-ba0d-f9223c6396b7"
steps,sklearn-kmeans
